In [2]:
import numpy as np
import tifffile
import glob
import skimage.util as util
import cv2
import dask
import pandas as pd
import napari
import plotly.express as px
import skimage as ski
import glob
import plotly.graph_objects as go

In [3]:
viewer = napari.Viewer()

# Using cellpose found objects, defunct

In [51]:
df = pd.read_csv('Cells.csv')
df['short_fname'] = df['fname'].str.split('\\').str[1]

In [53]:
fnames = glob.glob('1_stage_IB/*.png')
fnames.extend(glob.glob('2_stage_II/*.png'))
fnames.extend(glob.glob('3_stage_mid_III/*.png'))
fnames.extend(glob.glob('4_stage_late_III/*.png'))
#fnames.extend(glob.glob('junk/*.png'))

In [55]:
for fname in fnames:
    assigned_class = fname.split('/')[0]
    orig_fname = fname.split('/')[1].split('.')[0]+'.tif'
    orig_class = fname.split('/')[1].split('.')[1][0]
    orig_label = int(fname.split('/')[1].split('.')[1].split('_')[1])
    df.loc[(df['short_fname'] == orig_fname) & (df['label'] == orig_label) & (df['type'] == orig_class), 'assigned'] = assigned_class
df = df.dropna()

In [57]:
fnames = df['fname'].unique()

In [77]:
# Loop over all files
for fname in fnames:
    output_img_path = 'prelim_data/train/images/' + fname.split('\\')[-1]
    output_labels_path = 'prelim_data/train/labels/' + fname.split('\\')[-1].replace('.tif', '.txt')
    sub_df = df[df['fname'] == fname]
    img = ski.io.imread(fname.replace('\\','/'))
    width, height = img.shape[0:2]

    with open(output_labels_path, 'w') as f:
        # 1_stage_IB
        small_df = sub_df[sub_df['assigned'] == '1_stage_IB']
        for idx, row in small_df.iterrows():
            min_col, min_row, max_col, max_row = row['bbox-0'], row['bbox-1'], row['bbox-2'], row['bbox-3']

            # Calculate YOLO format values
            x_center = (min_col + max_col) / (2 * width)
            y_center = (min_row + max_row) / (2 * height)
            box_width = (max_col - min_col) / width
            box_height = (max_row - min_row) / height
            
            # Write to file (class is always 0 as mentioned)
            f.write(f"0 {x_center} {y_center} {box_width} {box_height}\n")
        
        # 2_stage_II
        small_df = sub_df[sub_df['assigned'] == '2_stage_II']
        for idx, row in small_df.iterrows():
            min_col, min_row, max_col, max_row = row['bbox-0'], row['bbox-1'], row['bbox-2'], row['bbox-3']

            # Calculate YOLO format values
            x_center = (min_col + max_col) / (2 * width)
            y_center = (min_row + max_row) / (2 * height)
            box_width = (max_col - min_col) / width
            box_height = (max_row - min_row) / height
            
            # Write to file (class is always 0 as mentioned)
            f.write(f"1 {x_center} {y_center} {box_width} {box_height}\n")

        # 3_stage_mid_III
        small_df = sub_df[sub_df['assigned'] == '3_stage_mid_III']
        for idx, row in small_df.iterrows():
            min_col, min_row, max_col, max_row = row['bbox-0'], row['bbox-1'], row['bbox-2'], row['bbox-3']

            # Calculate YOLO format values
            x_center = (min_col + max_col) / (2 * width)
            y_center = (min_row + max_row) / (2 * height)
            box_width = (max_col - min_col) / width
            box_height = (max_row - min_row) / height
            
            # Write to file (class is always 0 as mentioned)
            f.write(f"2 {x_center} {y_center} {box_width} {box_height}\n")

        # 4_stage_late_III
        small_df = sub_df[sub_df['assigned'] == '4_stage_late_III']
        for idx, row in small_df.iterrows():
            min_col, min_row, max_col, max_row = row['bbox-0'], row['bbox-1'], row['bbox-2'], row['bbox-3']

            # Calculate YOLO format values
            x_center = (min_col + max_col) / (2 * width)
            y_center = (min_row + max_row) / (2 * height)
            box_width = (max_col - min_col) / width
            box_height = (max_row - min_row) / height
            
            # Write to file (class is always 0 as mentioned)
            f.write(f"3 {x_center} {y_center} {box_width} {box_height}\n")

        f.close()
    cv2.imwrite(output_img_path, cv2.cvtColor(img, cv2.COLOR_RGB2BGR))


# Check Results in Napari

In [120]:
viewer = napari.Viewer()

In [121]:
fnames = glob.glob('new_data/train/images/*.tif')

In [141]:
fname = fnames[0]
fname

'new_data/train/images\\01_02_02.tif'

In [142]:
img = ski.io.imread(fname)
labels = np.loadtxt(fname.replace('images', 'labels').replace('.tif', '.txt'))

In [143]:
height, width = img.shape[0:2]

In [144]:
labels[:,1] = (labels[:,1] * width).astype(int)
labels[:,2] = (labels[:,2] * height).astype(int)
labels[:,3] = (labels[:,3] * width/2).astype(int)
labels[:,4] = (labels[:,4] * height/2).astype(int)

In [145]:
labels[:,1] = labels[:,1] - labels[:,3]
labels[:,2] = labels[:,2] - labels[:,4]
labels[:,3] = labels[:,1] + 2*labels[:,3]
labels[:,4] = labels[:,2] + 2*labels[:,4]

In [146]:
lst = []
for label in labels[labels[:,0]==0]:
    y1, x1, y2, x2 = label[1:]
    lst.append(np.array([[x1, y1], [x2, y1], [x2, y2], [x1, y2]]))
t0_rects = np.array(lst)

lst = []
for label in labels[labels[:,0]==1]:
    y1, x1, y2, x2 = label[1:]
    lst.append(np.array([[x1, y1], [x2, y1], [x2, y2], [x1, y2]]))
t1_rects = np.array(lst)

lst = []
for label in labels[labels[:,0]==2]:
    y1, x1, y2, x2 = label[1:]
    lst.append(np.array([[x1, y1], [x2, y1], [x2, y2], [x1, y2]]))
t2_rects = np.array(lst)

lst = []
for label in labels[labels[:,0]==3]:
    y1, x1, y2, x2 = label[1:]
    lst.append(np.array([[x1, y1], [x2, y1], [x2, y2], [x1, y2]]))
t3_rects = np.array(lst)

lst = []
for label in labels[labels[:,0]==4]:
    y1, x1, y2, x2 = label[1:]
    lst.append(np.array([[x1, y1], [x2, y1], [x2, y2], [x1, y2]]))
t4_rects = np.array(lst)

In [147]:
viewer.add_image(img)
if len(t0_rects)>0:
    viewer.add_shapes(t0_rects, shape_type='rectangle', edge_color='red', face_color='red', opacity=0.6, name='T0')
if len(t1_rects)>0:
    viewer.add_shapes(t1_rects, shape_type='rectangle', edge_color='green', face_color='green', opacity=0.6, name='T1')
if len(t2_rects)>0:
    viewer.add_shapes(t2_rects, shape_type='rectangle', edge_color='blue', face_color='blue', opacity=0.6, name='T2')
if len(t3_rects)>0:
    viewer.add_shapes(t3_rects, shape_type='rectangle', edge_color='yellow', face_color='yellow', opacity=0.6, name='T3')
if len(t4_rects)>0:
    viewer.add_shapes(t4_rects, shape_type='rectangle', edge_color='purple', face_color='purple', opacity=0.6, name='T4')

In [148]:
fname

'new_data/train/images\\01_02_02.tif'

# Export Manual

In [149]:
layer_names = [layer.name for layer in viewer.layers[1:]]

In [150]:
layer_names

['T0', 'T1', 'T3']

In [151]:
for layer in layer_names:
    rects = viewer.layers[layer].data
    np.save(fname.replace('.tif', f'_{layer}.npy'), rects)

In [152]:
fname

'new_data/train/images\\01_02_02.tif'